## Chapter 6　汎化性能を向上させる技術
---
注：
 - ６章の内容は手法の羅列のみなので、詳細や実装は別の資料を参考にすることを推奨する。
 - 数式の詳細や実装についてはすでに他のnotebookで扱ったので、ここでは深入りしていない。

---
### 6.1　汎化誤差の分解
汎化誤差は以下の３つに分解して考えることができる。

 - 近似誤差：モデルの表現力不足により、パラメータを増やすなどして自由度を上げると改善
 - 推定誤差：訓練データの偏りにより、訓練データを増やすか自由度を下げると改善
 - 最適化誤差：最適化アルゴリズムにより、別のアルゴリズムを試すと改善することがある

訓練データでの誤差が大ならば近似誤差や最適化誤差を考え、  
評価データや開発データでの誤差が大ならば推定誤差が大きいと判断できる。

---
### 6.2　推定誤差低減に効く手法
推定誤差を防ぐ手法は正則化（regularization）と呼ばれる。  
これらの手法は一般に
 - 訓練データの差が予測結果に大きく影響することを防ぐため、ばらつきを抑える
 - 事前知識を用いて特定の仮定を置く

という点が共通する。


---
#### L2正則化 （L2 regularization）
パラメータの値の大きさに罰則を与える正則化項を導入する。

\begin{align*}
L(\theta) + \frac{\lambda}{2}||\theta||^{2}
\end{align*}

ここで $\lambda$ は正則化の強さを調整するパラメータである。

正則化項の偏微分は

\begin{align*}
\frac{\partial}{\partial \theta} \frac{\lambda}{2}||\theta||^{2} = \lambda \theta
\end{align*}

であるから、勾配法での更新式は次の通り。

\begin{align*}
\theta^{(k+1)} &= \theta^{(k)} - \eta \partial L(\theta^{(k)}) - \eta \lambda \theta^{(k)} \\
&= (1-\eta \lambda) \theta^{(k)} - \eta \partial L(\theta^{(k)})
\end{align*}

式より、パラメータの値は $(1-\eta \lambda) \leq 1$ 倍されることがわかる。


---
#### early stopping
学習中に性能評価を実施して、改善が見られない場合に学習を早期終了することで過学習を防ぐ手法である。

---
#### 学習率減衰 （learning rate decay）
学習中に性能評価を実施して、改善が見られない場合に更新率を下げる手法である。  
　（注：この定義は著者により差があり、改善の有無を見ずに更新率を下げる手法について言われることのほうが多い）


---
#### パラメータ共有 （parameter sharing）
ニューラルネット内の複数の要素あるいは複数のネット間で同じパラメータを使用する手法の総称である。  
これは学習対象について何らかの共通点があることを仮定しており、  
手法の使用にあたっては事前知識を必要とすることに注意が必要である。

RNNにおいて入力単語と出力単語のパラメータ行列を共有するのが有名な適用例であり、  
性能が向上した例が報告されている。

マルチタスク学習でもしばしば適用があり、  
複数言語間翻訳でモデルの一部を共有することで性能が改善する例が報告されている。

パラメータ共有に類似した手法としてパラメータ結束（parameter tying）があり  
２つのパラメータの差に対して罰則を与えることで近いパラメータを学習させる。  
罰則としてはフロベニウスノルム（Frobenius norm）の２乗

\begin{align*}
||\mathbf{W}^{(a)} - \mathbf{W}^{(b)}||^{2}
\end{align*}

などを用いる。

---
#### 事前学習 （pre-training）
別の補助タスクで学習したパラメータを主タスクのニューラルネットの一部として使う手法である。  
このうち、学習するときの初期値として使い、主タスクの損失を減らすよう学習させるものを fine tuning という。

NLP分野では単語埋め込みベクトルが最も広く使われている例で、  
事前学習したパラメータで固定または初期化していることが多い。

例えば補助タスクとして一般的な大規模コーパスにより埋め込みベクトルを学習し  
主タスクとして目的の特定分野に特化したコーパスにより学習することで  
性能向上がみられることがしばしばある。

---
#### アンサンブル法 （ensemble method）
複数モデルを組み合わせて予測する手法で、  
コミッティマシン（committee machines）やモデル平均化法（model averaging）とも呼ばれる。

アンサンブル法はいわばモデルの多数決による予測で、  
単独モデルでの予測より正解する確率が高いことが期待できる。

類似のものとして、バギング（bootstrap aggregating, bagging）がある。

---
#### ドロップアウト （dropout）
訓練時に状態変数の一部をランダムに０に設定する手法である。

実装としては、ドロップアウトマスク $\zeta^{(l)} \in \{ 0,1 \}^{N^{(l)}}$ を用いて

\begin{align*}
\mathbf{h}^{(l)} = \zeta^{(l)} \odot \mathbf{h}^{(l-1)}
\end{align*}

の計算により出力を加工する。

Y.Gal and Z.Ghahramani, [A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](https://arxiv.org/abs/1512.05287) In proc. of NIPS, 2016.  
ではベイズ的解釈に基づき、ドロップアウトをパラメータ分布からサンプリング方法ととらえ、  
事後分布を変分近似しているとの解釈が与えられている。  

この解釈を元に提案された Monte-Carlo Dropout は  
予測時に複数の Dropout パターンを試して平均を取る手法で、  
単一モデルによる既存手法よりも良い精度を出した。

これらの Dropout 手法は、暗にアンサンブル法を適用しているものと解釈される。

---
### 6.3　最適化誤差低減に効く手法


#### 初期値設定
初期値が同じパラメータは勾配法を適用すると同じく更新されてしまうので、  
それぞれの値を違える必要がある。

一般にはXavierの初期値やHeの初期値が頻用されている。

IRNNはRNNのパラメータ行列（の一部）を単位行列で初期化する手法である。これは

\begin{align*}
\mathbf{h}_{t}^{(l)} = a^{(l)} (\tilde{\mathbf{W}}^{(l)}\mathbf{h}_{t}^{(l-1)} + \mathbf{h}_{t-1}^{(l)} + \mathbf{b}^{(l)})
\end{align*}

のように、ある層から次の層へのショートカットを形成することに等しい。

また、忘却ゲートにおけるバイアスパラメータの初期値を１など大きな値にすることも  
同様にショートカット形成としての効果がある。

---
#### カリキュラム学習 （curriculum learning）
学習過程を複数段階に分け、簡単な概念からはじめて  
徐々に複雑な概念ないしタスクを学習させていく手法である。

タスクの難易度について仮定または事前知識が必要なものの  
複数の分野で有用性が報告されている。

---
#### 正規化
勾配法ではパラメータ同士の関係を考慮しないため、パラメータ間に依存性があると学習が安定しないことがある。

ニュートン法は有効であることが知られているが、  
パラメータ数の２乗に比例して計算量が増加するために負荷が高い。

バッチ正規化（batch normalization）はよく使われる手法で、  
ミニバッチ内で計算した平均と分散を用いて入力を正規化する。  
予測時には訓練データ中の複数のミニバッチについて求めた平均と分散とで正規化を行う。

類似手法に層正規化（layer normalization）があり、各層の隠れ状態ベクトルの要素集合について正規化する。

---
### 6.4　超パラメータ選択
ハイパーパラメータの選択にはグリッド探索やランダム探索がよく用いられる。  
この２者について現在のところ優劣を示す証拠はないので、あなたの信仰に従って選んでよい。

ベイズ的最適化（Bayesian optimization; BO）はガウス過程による回帰を用いて  
説明変数と目的変数との間で回帰モデルを構築し、  
よりよい目的変数の値を与える可能性の高い説明変数を見つける手法である。

BOをハイパーパラメータ選択に適用することで、  
より良い性能を与える可能性が高く、かつ評価の不確実性が高い候補ほど高い確率で調査できるため  
効率の良い探索と選択ができる。

---
次のnotebookへ続く。